# Transfer learning of a deep CNN on XPS data on Google Colab

In this notebook, we will use transfer learning for training a deep convolutional network on new XPS spectra using a pre-trained model. Using Google Colab.

## Setup

### Mount google drive, change working directory

In [ ]:
# Mount drive
from google.colab import drive
import os

drive.mount('/content/drive')

# Change working path
os.chdir('/content/drive/My Drive/app')

### Install packages and import modules

In [ ]:
# Install packages
#!pip install tensorflow==2.1.0 #only for now since the progress bar looks bad
!pip install python-docx

# Import standard modules and magic commands
import datetime
import numpy as np
import pytz
import importlib

# Magic commands
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Disable tf warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

### Load custom modules

In [ ]:
try:
    import importlib
    importlib.reload(classifier)
    importlib.reload(clfutils)
    print('Modules were reloaded.')
except:
    import network.classifier as classifier
    import network.utils as clfutils
    print('Modules were loaded.')

### Setting up the parameters & folder structure

In [ ]:
np.random.seed(502)
time =  datetime.datetime.now().astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y%m%d_%Hh%Mm")
data_name = 'Fe_single_4_classes'

label_values = ['Fe metal', 'FeO', 'Fe3O4', 'Fe2O3']

clf = classifier.Classifier(time = time,
                         data_name = data_name,
                         labels = label_values)

### Load and inspect the data

In [ ]:
input_filepath = r'/content/drive/My Drive/app/datasets/20200605_iron_single_small.h5'
train_test_split = 0.2
train_val_split = 0.2
no_of_examples = 1000

X_train, X_val, X_test, y_train, y_val, y_test = clf.load_data_preprocess(input_filepath = input_filepath,
                                                                          no_of_examples = no_of_examples,
                                                                          train_test_split = train_test_split,
                                                                          train_val_split = train_val_split)
# Check how the examples are distributed across the classes:
class_distribution = clf.check_class_distribution()
clf.plot_class_distribution()
clf.plot_random(no_of_spectra = 9, dataset = 'train')  

## Load pre-trained model

### Load model without classification layers

In [ ]:
clf.load_model(model_path = r'/content/drive/My Drive/app/saved_models/20200626_10h04m_Fe_single_4_classes',
               drop_last_layers = 2)
clf.summary()

### Add new classification layers

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

dense_1 = Dense(2000, activation = 'relu')(clf_2.model.layers[-1].output)
dense_2 = Dense(clf.num_classes, activation = 'softmax')(dense_1)
clf_2.model = Model(inputs = clf_2.model.input,
                  outputs = dense_2,
                  name = 'Custom_CNN_Sub')
clf_2.summary()

### Compile and summarize the model

In [ ]:
from tensorflow.keras.optimizers import Adam
learning_rate = 1e-05
optimizer = Adam(learning_rate = learning_rate) 

clf.model.compile(loss = 'categorical_crossentropy',
                  optimizer = optimizer, 
                  metrics = ['accuracy'])


# Plot summary and save model plot.
clf.summary()
clf.save_and_print_model_image()

### Training

In [ ]:
epochs = 10
batch_size = 32

hist = clf.train(checkpoint = True,
                 early_stopping = False,
                 tb_log = True, 
                 csv_log = True,
                 epochs = epochs, 
                 batch_size = batch_size,
                 verbose = 1)

### Graphs

In [ ]:
dir_name = clf.time + '_' + clf.data_name
graph = clfutils.TrainingGraphs(clf.history, dir_name) 

### Evaluation on test data

In [ ]:
score = clf.evaluate()
test_loss, test_accuracy = score[0], score[1]
print('Test loss: ' + str(np.round(test_loss, decimals=3)))
print('Test accuracy: ' + str(np.round(test_accuracy, decimals=3)))

### Prediction on test data

In [ ]:
pred_train, pred_test = clf.predict()
pred_train_classes, pred_test_classes = clf.predict_classes()

### Show some predictions

#### Training data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'train', with_prediction = True)  

#### Test data

In [ ]:
clf.plot_random(no_of_spectra = 6, dataset = 'test', with_prediction = True)  

### Saving data

In [ ]:
clf.save_model()
clf.save_hyperparams()
clf.shelve_results(full = False)

### Report

In [ ]:
dir_name = clf.time + '_' + clf.data_name
rep = clfutils.Report(dir_name)  
rep.write()   

### Save output of notebook

In [ ]:
from IPython.display import Javascript, display
from nbconvert import HTMLExporter

def save_notebook():
    display(Javascript("IPython.notebook.save_notebook()"), include=['application/javascript'])

def output_HTML(read_file, output_file):
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

import time

save_notebook()
time.sleep(3)
current_file = 'transfer_learning.ipynb'
output_file = os.path.join(clf.log_dir,'transfer_learning_out.html')
output_HTML(current_file, output_file)